In [71]:
from os import listdir
texfiles = [f for f in listdir(".") if f.endswith(".tex")]

In [142]:
filename = texfiles[3]
print filename

classgroup.tex


In [151]:
texfile = open(filename).read()

In [152]:
def lines_in_env(environment,lines):
    """
    Given some lines of tex code and an environment (such as "enumerate" or "align")
    this returns the index of the first line that starts with "\\begin{environment}"
    and the number of lines until "\\end{environment}".
    
    Note: lines[start:start+length] should include both the begin and end lines.
    """
    is_start = lambda line: line.strip().startswith("\\begin{%s}" % environment)
    is_end   = lambda line: line.strip().startswith("\\end{%s}"   % environment)
    start    = next(i for i,line in enumerate(lines) if is_start(line))
    length   = next(i+1 for i,line in enumerate(lines[start:]) if is_end(line))
    return start,length

In [153]:
def get_cells(tex):
    """
    Given a string of tex, returns the lines of {sagecell}
    environments grouped together by {sagecode} environments
    respecting "%link" commands.
    """
    lines = tex.splitlines()
    blocks = []
    i = 0
    while True:
        try:
            start,length = lines_in_env("sagecode",lines[i:])
            blocks.append({
                "start": i+start,
                "length": length,
            })
            i += start + length
        except StopIteration:
            break
    cells = [[]]
    for block in blocks:
        start,length = block["start"],block["length"]
        j = 0
        while True:
            try:
                cell_start,cell_length = lines_in_env("sagecell",lines[start+j:start+length])
                out_start,out_length = lines_in_env("sageout",lines[start+j:start+length])
                assert cell_start + cell_length == out_start
                if not lines[start+j+cell_start].strip().endswith("%skip"):
                    cells[-1].append({
                        "source": {"start":cell_start+j+start,"length":cell_length},
                        "output": {"start":out_start+j+start,"length":out_length},
                    })
                j += cell_length + out_length
            except StopIteration:
                break
        if not lines[start+length-1].strip().endswith("%link"):
            cells.append([])
    return [a for a in cells if len(a) > 0]

In [154]:
def generate_tests(tex):
    """
    Given a string of tex, this returns a sage file
    containing all the code with comments for line numbers.
    This can be used to test the code samples.
    """
    lines = tex.splitlines()
    output = "\n\n"
    cells = get_cells(tex)
    for block in cells:
        output += "\"\"\"\n"
        for cell in block:
            source_start,source_length = cell["source"]["start"],cell["source"]["length"]
            output += "sage: # From line %s" % (source_start+1) + "\n"
            for i in range(source_start + 1,source_start + source_length - 1):
                if lines[i].startswith("\t") or lines[i].startswith("  ") or lines[i-1].strip().endswith("\\"):
                    prefix = "....: "
                else:
                    prefix = "sage: "
                output += prefix + lines[i].replace("\t","    ") + "\n"
            out_start,out_length = cell["output"]["start"],cell["output"]["length"]
            for i in range(out_start + 1,out_start + out_length - 1):
                output += lines[i] + "\n"
        output += "\"\"\"\n\n"
    return output

In [155]:
testfile = generate_tests(texfile)

In [156]:
print testfile



"""
sage: # From line 83
sage: K = QQ[sqrt(10)]; K
Number Field in sqrt10 with defining polynomial x^2 - 10
sage: # From line 89
sage: B = K.minkowski_bound(); B
sqrt(10)
sage: # From line 95
sage: B.n()
3.16227766016838
"""

"""
sage: # From line 150
sage: K.<sqrt10> = QQ[sqrt(10)]; K
Number Field in sqrt10 with defining polynomial x^2 - 10
sage: # From line 156
sage: G = K.class_group(); G
Class group of order 2 with structure C2 of Number Field ...
sage: # From line 162
sage: G.0
Fractional ideal class (3, sqrt10 + 1)
sage: # From line 168
sage: G.0^2
Trivial principal fractional ideal class
sage: # From line 174
sage: G.0 == G( (3, 2 + sqrt10) )
True
"""

"""
sage: # From line 521
sage: for d in [2..1000]:
....:     if is_fundamental_discriminant(d):
....:         h = QuadraticField(d, 'a').class_number()
....:         if h == 1:
....:             print d
5 8 12 13 17 21 24 28 29 33 37 41 44 53 56 57 61 69
73 76 77 88 89 92 93 97 101 109 113 124 129 133 137
141 149 152 157 161 17

In [157]:
f = open("/tmp/test.sage","wt")
f.write(testfile)
f.close()

In [158]:
import os
print os.popen('sage -t /tmp/test.sage').read()

too few successful tests, not using stored timings
Running doctests with ID 2018-08-06-10-21-25-9765580b.
Git branch: develop
Using --optional=mpir,python2,sage
Doctesting 1 file.
sage -t /tmp/test.sage
**********************************************************************
File "/tmp/test.sage", line 35, in test
Failed example:
    for d in [2..1000]:
        if is_fundamental_discriminant(d):
            h = QuadraticField(d, 'a').class_number()
            if h == 1:
                print d
Exception raised:
    Traceback (most recent call last):
      File "/home/t/SageMath/local/lib/python2.7/site-packages/sage/doctest/forker.py", line 551, in _run
        self.compile_and_execute(example, compiler, test.globs)
      File "/home/t/SageMath/local/lib/python2.7/site-packages/sage/doctest/forker.py", line 959, in compile_and_execute
        compiled = compiler(example)
      File "/home/t/SageMath/local/lib/python2.7/site-packages/sage/doctest/forker.py", line 549, in <lambda>
       